In [19]:
from typing import List, Tuple
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
from glob import glob

import pandas as pd
import numpy as np
import dill

from maze_io import load_maze, string_maze, add_padding
from maze_env import MazeEnv, MazeAction
from agent import Agent, manhattan, euclid, zero_heuristic
from navmesh_agent import NavmeshAgent
from agent_utils import Stats
from general_utils import get_optimal_path_length

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
def run_single_maze(start_loc, exit_loc, maze, agentClass, heuristic, agent_verbose=False) -> Stats:
    env = MazeEnv(start_loc, exit_loc, maze)
    agent = agentClass(env, heuristic=heuristic, verbose=agent_verbose)
    agent.run()
    return agent.statistics

def prepare_maze_from_file(path):
    with path.open(mode='r') as f:
        padded_maze_tuple = add_padding(*load_maze(f))
    return padded_maze_tuple

In [21]:
n_times = 100  # how many times to repeat each maze-agent settings combination

In [22]:
inpaths = list(map(Path, glob('./mazes/*.txt')))
maze_names = [path.stem for path in inpaths]

# Compute or load A* Agent statistics
Beware, takes a very long time (2h+), load them if any available

In [23]:
def run_a_star_experiments(each_n_times: int,
                           inpaths: List[Path],
                           numpy_seed=123,
                           save_to_file=False) -> dict:
    """
    Function for running A* experiments for all heuristics (None, Euclidean dist, Manhattan dist)
    :param each_n_times: How many times to run each maze/heuristic combination
    :param inpaths: List of paths to each maze file
    :param numpy_seed: The same seed is used before running each maze/heuristic combination each_n_times times,
     so that different agents experience the same environment behaviour over the experiments.
    :param save_to_file: Whether to save the results to file
    :return: A nested dictionary with results,
     examine the function to see structure,
     but it is recommended to just use the following function
     to transform output of this function into a DataFrame
    """
    data = {}
    heuristics = {'None': zero_heuristic,
                  'Euclid': euclid,
                  'Manhattan': manhattan}
    for maze_num, path in enumerate(inpaths, start=1):
        maze_name = path.stem
        maze_data = prepare_maze_from_file(path)
        optimal_length = get_optimal_path_length(*maze_data)
        data[maze_name] = {"optimal": optimal_length}

        for heuristic_name, heuristic in heuristics.items():
            print(f'\r#{maze_num}/{len(inpaths)}: {maze_name} | {heuristic_name}         ', end="")
            np.random.seed(numpy_seed)
            data[maze_name][heuristic_name] = [run_single_maze(*maze_data,
                                                               Agent,
                                                               heuristic=heuristic,
                                                               agent_verbose=False)
                                               for _ in range(each_n_times)]

    if save_to_file:
        with open('./a_star_dict_data.dill', mode='wb') as f:
            dill.dump(data, f)

    print("\nDone")
    return data


In [24]:
load_astar_from_file = True  # if false, compute them
if load_astar_from_file:
    with open('./a_star_data_seeded.dill', mode='rb') as f:
        astar_data = dill.load(f)
else:
    astar_data = run_a_star_experiments(n_times, inpaths, save_to_file=False)

## Create A* dataframe

In [25]:
def create_dataframe_from_a_star_dict(astar_data: dict, maze_names: List[str]) -> pd.DataFrame:
    dfs = []
    for name in maze_names:
        cur_opt = astar_data[name]["optimal"]
        for heur, cur_stats in astar_data[name].items():
            if heur != "optimal":
                tuplized = [(np.sum(stats.expanded_nodes),
                             stats.replan_count,
                             stats.steps)
                            for stats in cur_stats]
                cur_expanded, cur_replans, cur_steps = zip(*tuplized)
                cur_dict = {"maze": name,
                            "heuristic": heur,
                            "optimal_steps": cur_opt,
                            "steps_taken": cur_steps,
                            "expanded_nodes": cur_expanded,
                            "replan_count": cur_replans}
                dfs.append(pd.DataFrame(cur_dict))
    df = pd.concat(dfs, ignore_index=True)
    df["maze"] = df["maze"].astype('category')
    df["heuristic"] = df["heuristic"].astype('category')
    return df

In [26]:
astar_df = create_dataframe_from_a_star_dict(astar_data, maze_names)

# Create Navmesh Agent statistics
Is fairly quick, under a minute

In [27]:
def run_navmesh_experiments(each_n_times: int, inpaths: List[Path], numpy_seed=123) -> Tuple[dict, dict]:
    data_navmesh = {}
    optimal_paths = {}
    for maze_num, path in enumerate(inpaths, start=1):
        maze_name = path.stem
        maze_data = prepare_maze_from_file(path)
        optimal_length = get_optimal_path_length(*maze_data)
        optimal_paths[maze_name] = optimal_length
        print(f'\r#{maze_num}/{len(inpaths)}: {maze_name}         ', end="")
        np.random.seed(numpy_seed)
        data_navmesh[maze_name] = [run_single_maze(*maze_data, NavmeshAgent, heuristic=None, agent_verbose=False)
                                   for _ in range(each_n_times)]
    print("\nDone")
    return data_navmesh, optimal_paths

## Create Navmesh Agent dataframe

In [28]:
def create_dataframe_from_navmesh_dict(navmesh_data_dict: dict, optimal_pathlen_dict: dict, maze_names: List[str]) -> pd.DataFrame:
    dataframes = []
    for name in maze_names:
        cur_maze_stats = navmesh_data_dict[name]
        cur_opt = optimal_pathlen_dict[name]
        tuplized = [(np.sum(stats.expanded_nodes),
                     1,
                     stats.steps) for stats in cur_maze_stats]
        cur_expanded, cur_replans, cur_steps = zip(*tuplized)
        cur_dict = {"maze": name,
                    "optimal_steps": cur_opt,
                    "steps_taken": cur_steps,
                    "expanded_nodes": cur_expanded,
                    "replan_count": cur_replans}
        dataframes.append(pd.DataFrame(cur_dict))
    df = pd.concat(dataframes, ignore_index=True)
    df["maze"] = df["maze"].astype('category')
    return df

In [67]:
nav_df = create_dataframe_from_navmesh_dict(*run_navmesh_experiments(n_times, inpaths), maze_names)

#25/25: maze-7-5          
Done


In [68]:
def create_everything_dataframe(navmesh_df: pd.DataFrame, astar_df: pd.DataFrame) -> pd.DataFrame:
    nav_df_copy = navmesh_df.copy()
    nav_df_copy['heuristic'] = "Navmesh"
    all_df = pd.concat([astar_df, nav_df_copy], ignore_index=True)
    all_df.rename(columns={"heuristic": "agent"}, inplace=True)
    all_df.agent = all_df.agent.astype('category')
    all_df.agent.cat.rename_categories({"None": "Dijkstra",
                                        "Manhattan": "A*Manhattan",
                                        "Euclid": "A*Euclid"},
                                       inplace=True)
    return all_df

In [71]:
all_df = create_everything_dataframe(nav_df, astar_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   maze            10000 non-null  category
 1   agent           10000 non-null  category
 2   optimal_steps   10000 non-null  int64   
 3   steps_taken     10000 non-null  int64   
 4   expanded_nodes  10000 non-null  int64   
 5   replan_count    10000 non-null  int64   
dtypes: category(2), int64(4)
memory usage: 333.2 KB


In [70]:
all_df.to_csv("all_dataframe.csv", index=False)

In [ ]:
all_df = pd.read_csv("all_dataframe.csv", dtype={"maze": "category", "agent": "category"})

# Various statistics

In [30]:
nav_df_sel = nav_df.groupby('maze').agg(optimal_steps=('optimal_steps', max),
                           mean_steps=('steps_taken', np.mean),
                           std_steps=('steps_taken', np.std),
                           min_steps=('steps_taken', min),
                           max_steps=('steps_taken', max),
                           expanded_nodes=('expanded_nodes', max),
                           replan_count=('replan_count', max))
nav_df_sel

,optimal_steps,mean_steps,std_steps,min_steps,max_steps,expanded_nodes,replan_count
maze,,,,,,,
maze-12-1,46,85.28,11.745689,54,113,339,1
maze-12-2,46,83.68,11.425543,60,121,350,1
maze-12-3,54,99.80,12.966935,72,132,356,1
maze-12-4,46,85.21,12.361798,54,119,356,1
maze-12-5,46,83.30,12.464495,61,138,351,1
maze-25-1,98,183.96,18.109052,145,231,1534,1
maze-25-2,104,193.04,20.271351,145,248,1556,1
maze-25-3,98,181.77,19.486776,139,242,1547,1
maze-25-4,98,184.21,18.770434,136,229,1542,1


In [31]:
astar_df.groupby(['maze', 'heuristic']).mean()

optimal_steps  steps_taken  expanded_nodes  replan_count
maze      heuristic                                                          
maze-12-1 Euclid              46.0        85.87         1572.67         13.25
          Manhattan           46.0        85.28          930.86         12.49
          None                46.0        85.28         2984.55         12.43
maze-12-2 Euclid              46.0        82.97         1782.60         14.02
          Manhattan           46.0        83.68          990.54         14.12
...                            ...          ...             ...           ...
maze-7-4  Manhattan           26.0        50.48          299.40          9.21
          None                26.0        50.02          784.90          8.84
maze-7-5  Euclid              26.0        46.04          440.61          8.28
          Manhattan           26.0        47.78          403.03          8.47
          None                26.0        47.78          797.06          8.47

[75 rows x 4 columns]

In [86]:
all_df_sel = all_df[all_df.maze.str.contains("-50-")].groupby(['maze', 'agent'], observed=True).agg(optimal_steps=('optimal_steps', max),
                                      min_steps=('steps_taken', min),
                                      mean_steps=('steps_taken', np.mean),
                                      max_steps=('steps_taken', max),
                                      std_steps=('steps_taken', np.std),
                                      min_expanded=('expanded_nodes', min),
                                      mean_expanded=('expanded_nodes', np.mean),
                                      max_expanded=('expanded_nodes', max),
                                      std_expanded=('expanded_nodes', np.std),
                                      min_plan=('replan_count', min),
                                      mean_plan=('replan_count', np.mean),
                                      max_plan=('replan_count', max),
                                      std_plan=('replan_count', np.std))

In [85]:
all_df[all_df.maze.str.contains("-50-")].groupby(['maze'], observed=True).mean()

,optimal_steps,steps_taken,expanded_nodes,replan_count
maze,,,,
maze-50-1,198.0,368.2725,109516.6600,45.1475
maze-50-2,204.0,380.2275,112208.6075,43.9350
maze-50-3,198.0,369.2175,100619.6850,44.1150
maze-50-4,198.0,366.1900,105085.0350,42.6925
maze-50-5,202.0,372.8650,108644.0300,44.6075
